<a href="https://colab.research.google.com/github/ponOlgap/OLga/blob/main/nlp_preprocessing(%D0%BF%D1%80%D0%B5%D0%B4%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предварительная обработка текста на Python


Устанавливаем библиотеку pymorphy2

In [ ]:
!pip install pymorphy2[fast] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 9.7 MB/s 
     |████████████████████████████████| 371 kB 55.2 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp38-cp38-linux_x86_64.whl size=927571 sha256=af2e1d4fbf2c5ac40c21ff07672979196c7cd90fd730f544b3fb9ceff4ecc370
  Stored in directory: /root/.cache/pip/wheels/1c/e6/8f/313a7ccc57b29a7affb7205664277a1d5ebe73bf600a69a615
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0df5c80f96444a4228d513895f94790cfe3bcbc164e7eb2bb25cb5bded147c42
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built DAWG docopt


Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Загружаем набор данных

In [ ]:
!wget https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1 -O banks.csv

--2022-12-29 16:58:09--  https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/a9r0b2yj3vqvi13/banks.csv [following]
--2022-12-29 16:58:10--  https://www.dropbox.com/s/dl/a9r0b2yj3vqvi13/banks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfb22ac42e60f7f2f70d89f3812.dl.dropboxusercontent.com/cd/0/get/BziM_Z2B_pUQB6cFcHJMUr4JNsYP-DWOcmsdLYG_dOGeBCCfFmj7Dl1vh4BkLa3cWrt5dylh4T_UBZq97SsFbqSoO83kZdV75OPCr3eItToY7Jx0fB4BjkvzvONIGVZR2K94rvWaNykmBkmVxIhI-zw2kYFUgxsMtx5o_FT-BvbG6w/file?dl=1# [following]
--2022-12-29 16:58:10--  https://ucfb22ac42e60f7f2f70d89f3812.dl.dropboxusercontent.com/cd/0/get/BziM_Z2B_pUQB6cFcHJMUr4JNsYP-DWOcmsdLYG_dOGeBCCfFmj7Dl1vh4BkLa3cWrt5dylh4T_UBZq97SsFbqS

In [ ]:
!head banks.csv

idx	Score	Text
0	Positive	В Альфа-Банке работает замечательная девушка - Ильясова Орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время моего прихода, помогла оформить кредит в размере 1млн рублей, когда я пришла с партнером (передавать ей полученный кредит за покупаемый мною авто), специалист Ильясова Орна помогла нам вывести всю сумму в один день (а это было непросто), так что сделка состоялась и все остались довольны! Моя знакомая в конце всего добавила: Теперь я  поняла, почему мы пришли в это отделение, к этой девушке..Побольше бы таких замечательных специалистов! Приобретать программу здоровье, и вообще все, что связано с Альфа-Банком, теперь буду только у нее!
1	Negative	Оформляя рассрочку в м. Видео в меге тёплый стан. Попросила менеджера, чтобы заявка была без страховки. В итоге, когда подписывала договор, в договоре оказались лишние почти 6 тысяч, когда менеджера стала спрашивать что это за

## Загружаем данные в память


In [ ]:
banks = pd.read_csv('banks.csv', sep='\t', index_col='idx');

In [ ]:
banks

,Score,Text
idx,,
0,Positive,В Альфа-Банке работает замечательная девушка -...
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,Positive,Очень порадовала оперативность работы в банке....
3,Negative,Имела неосторожность оформить потреб. кредит в...
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...


## Токенизация текста

In [ ]:
text = banks.iloc[0]['Text']

In [ ]:
text

'В Альфа-Банке работает замечательная девушка - Ильясова Орна, вежливая, отзывчивая, действительно участвует в запросе клиента, я приходила к ней подряд ровно три дня, каждый день она помнила время моего прихода, помогла оформить кредит в размере 1млн рублей, когда я пришла с партнером (передавать ей полученный кредит за покупаемый мною авто), специалист Ильясова Орна помогла нам вывести всю сумму в один день (а это было непросто), так что сделка состоялась и все остались довольны! Моя знакомая в конце всего добавила: Теперь я \xa0поняла, почему мы пришли в это отделение, к этой девушке..Побольше бы таких замечательных специалистов! Приобретать программу здоровье, и вообще все, что связано с Альфа-Банком, теперь буду только у нее!'

In [ ]:
tokens = word_tokenize(text.lower())

In [ ]:
tokens

['в',
 'альфа-банке',
 'работает',
 'замечательная',
 'девушка',
 '-',
 'ильясова',
 'орна',
 ',',
 'вежливая',
 ',',
 'отзывчивая',
 ',',
 'действительно',
 'участвует',
 'в',
 'запросе',
 'клиента',
 ',',
 'я',
 'приходила',
 'к',
 'ней',
 'подряд',
 'ровно',
 'три',
 'дня',
 ',',
 'каждый',
 'день',
 'она',
 'помнила',
 'время',
 'моего',
 'прихода',
 ',',
 'помогла',
 'оформить',
 'кредит',
 'в',
 'размере',
 '1млн',
 'рублей',
 ',',
 'когда',
 'я',
 'пришла',
 'с',
 'партнером',
 '(',
 'передавать',
 'ей',
 'полученный',
 'кредит',
 'за',
 'покупаемый',
 'мною',
 'авто',
 ')',
 ',',
 'специалист',
 'ильясова',
 'орна',
 'помогла',
 'нам',
 'вывести',
 'всю',
 'сумму',
 'в',
 'один',
 'день',
 '(',
 'а',
 'это',
 'было',
 'непросто',
 ')',
 ',',
 'так',
 'что',
 'сделка',
 'состоялась',
 'и',
 'все',
 'остались',
 'довольны',
 '!',
 'моя',
 'знакомая',
 'в',
 'конце',
 'всего',
 'добавила',
 ':',
 'теперь',
 'я',
 'поняла',
 ',',
 'почему',
 'мы',
 'пришли',
 'в',
 'это',
 'отделен

## Фильтрация знаков препинания

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']

In [ ]:
only_words = []
for token in tokens:
    if token not in punctuation_marks:
        only_words.append(token)

In [ ]:
only_words

['в',
 'альфа-банке',
 'работает',
 'замечательная',
 'девушка',
 'ильясова',
 'орна',
 'вежливая',
 'отзывчивая',
 'действительно',
 'участвует',
 'в',
 'запросе',
 'клиента',
 'я',
 'приходила',
 'к',
 'ней',
 'подряд',
 'ровно',
 'три',
 'дня',
 'каждый',
 'день',
 'она',
 'помнила',
 'время',
 'моего',
 'прихода',
 'помогла',
 'оформить',
 'кредит',
 'в',
 'размере',
 '1млн',
 'рублей',
 'когда',
 'я',
 'пришла',
 'с',
 'партнером',
 'передавать',
 'ей',
 'полученный',
 'кредит',
 'за',
 'покупаемый',
 'мною',
 'авто',
 'специалист',
 'ильясова',
 'орна',
 'помогла',
 'нам',
 'вывести',
 'всю',
 'сумму',
 'в',
 'один',
 'день',
 'а',
 'это',
 'было',
 'непросто',
 'так',
 'что',
 'сделка',
 'состоялась',
 'и',
 'все',
 'остались',
 'довольны',
 'моя',
 'знакомая',
 'в',
 'конце',
 'всего',
 'добавила',
 'теперь',
 'я',
 'поняла',
 'почему',
 'мы',
 'пришли',
 'в',
 'это',
 'отделение',
 'к',
 'этой',
 'девушке',
 'побольше',
 'бы',
 'таких',
 'замечательных',
 'специалистов',
 'при

## Приводим слова к нормальной форме

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
lemmas = []
for token in only_words:
    lemmas.append(morph.parse(token)[0].normal_form)

In [ ]:
lemmas

['в',
 'альфа-банк',
 'работать',
 'замечательный',
 'девушка',
 'ильясова',
 'орный',
 'вежливый',
 'отзывчивый',
 'действительно',
 'участвовать',
 'в',
 'запрос',
 'клиент',
 'я',
 'приходить',
 'к',
 'она',
 'подряд',
 'ровно',
 'три',
 'день',
 'каждый',
 'день',
 'она',
 'помнить',
 'время',
 'мой',
 'приход',
 'помочь',
 'оформить',
 'кредит',
 'в',
 'размер',
 '1млн',
 'рубль',
 'когда',
 'я',
 'прийти',
 'с',
 'партнёр',
 'передавать',
 'она',
 'получить',
 'кредит',
 'за',
 'покупать',
 'я',
 'авто',
 'специалист',
 'ильясова',
 'орный',
 'помочь',
 'мы',
 'вывести',
 'весь',
 'сумма',
 'в',
 'один',
 'день',
 'а',
 'это',
 'быть',
 'непросто',
 'так',
 'что',
 'сделка',
 'состояться',
 'и',
 'всё',
 'остаться',
 'довольный',
 'мой',
 'знакомый',
 'в',
 'конец',
 'весь',
 'добавить',
 'теперь',
 'я',
 'понять',
 'почему',
 'мы',
 'прислать',
 'в',
 'это',
 'отделение',
 'к',
 'этот',
 'девушка',
 'большой',
 'бы',
 'такой',
 'замечательный',
 'специалист',
 'приобретать',
 'п

## Удаление стоп слов

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words("russian")

In [ ]:
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [ ]:
filtered_words = []
for token in lemmas:
    if token not in stop_words:
        filtered_words.append(token)

In [ ]:
filtered_words

['альфа-банк',
 'работать',
 'замечательный',
 'девушка',
 'ильясова',
 'орный',
 'вежливый',
 'отзывчивый',
 'действительно',
 'участвовать',
 'запрос',
 'клиент',
 'приходить',
 'подряд',
 'ровно',
 'день',
 'каждый',
 'день',
 'помнить',
 'время',
 'приход',
 'помочь',
 'оформить',
 'кредит',
 'размер',
 '1млн',
 'рубль',
 'прийти',
 'партнёр',
 'передавать',
 'получить',
 'кредит',
 'покупать',
 'авто',
 'специалист',
 'ильясова',
 'орный',
 'помочь',
 'вывести',
 'весь',
 'сумма',
 'день',
 'это',
 'непросто',
 'сделка',
 'состояться',
 'всё',
 'остаться',
 'довольный',
 'знакомый',
 'конец',
 'весь',
 'добавить',
 'понять',
 'почему',
 'прислать',
 'это',
 'отделение',
 'девушка',
 'большой',
 'замечательный',
 'специалист',
 'приобретать',
 'программа',
 'здоровье',
 'вообще',
 'всё',
 'связать',
 'альфа-банк']

## Функция предварительной обработки

In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [ ]:
banks[0:5].apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

idx
0    [альфа-банк, работать, замечательный, девушка,...
1    [оформлять, рассрочка, м., видео, мег, тёплый,...
2    [очень, порадовать, оперативность, работа, бан...
3    [иметь, неосторожность, оформить, потреба, кре...
4    [небольшой, предыстория, найти, сайт, mdm, бан...
dtype: object

In [ ]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан..."
...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация..."
